In [1]:
!pip3 install pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyomo: filename=Pyomo-6.6.1-py2.py3-none-any.whl size=3206872 sha256=b28097014b5b7f81787eca064fb7647c523653525b6ccf42d22f7da8273dcea5
  Stored in directory: /Users/Apiux/Library/Caches/pip/wheels/9f/27/53/0e954d2b2f5734e1ed26f78c95b00e29533fca902819818f0f
Successfully built pyomo


In [2]:
conda install -c conda-forge glpk

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0



## Package Plan ##

  environment location: /Users/Apiux/anaconda3

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.5.7   |       h8857fd0_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    glpk-5.0                   |       h3cb5acd_0         1.0 MB  conda-forge
    openssl-1.1.1u             |       h8a1eda9_0         1.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2

In [4]:
import numpy as np
import pyomo.environ as pyo
np.random.seed(10)

In [5]:
J=range(10) 
U= range(5)
print("conjunto J:", [j for j in J])
print("conjunto U:", [u for u in U])

conjunto J: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
conjunto U: [0, 1, 2, 3, 4]


In [7]:
CB=100 #costo por no satisfacer la demanda
P=4 #puntos verdes a abrir
Dist_max = 60 #distancia maxima que recorren
f= np.round(np.random.uniform(10,20,len(J)),0) #costo localizar un punto verde
dist=np.round(np.random.uniform(10,200,(len(U),len (J))),0) #matriz de distancia
dda=np.round(np.random.uniform(5,20,len(U)),0) #DEMANDA

print (f)

[18. 10. 16. 17. 15. 12. 12. 18. 12. 11.]


In [8]:
print (dist)

[[140. 191.  11. 107. 164. 126. 147.  65. 184. 146.]
 [113.  37.  81. 138.  94.  92. 127. 107. 134. 124.]
 [163. 109. 183.  71.  27.  67.  32. 167.  19. 129.]
 [114. 166.  48. 173.  77. 153.  66. 178.  72.  41.]
 [ 85.  28. 166.  39.  83. 189. 198.  97. 167.  58.]]


In [11]:
print (dda)

[14. 19. 13. 14.  6.]


In [62]:
#crear modelo de optimizacion
m = pyo.ConcreteModel()

#variables
m.y = pyo.Var(J,within=pyo.Binary)
m.x = pyo.Var(U,J,within=pyo.Binary)

#funcion objetivo

costo_apertura =sum(f[j]*m.y[j] for j in J)
costo_relocalizacion= 0
costo_social= 0 #microbasural
costo_no_demanda= CB*(dda[u]*sum((1-sum(m.x[u,j] for j in J)) for u in U)) #NO SATISFACER LA DEMANDA


m.fo = pyo.objective(expr = costo_apertura + 
                  costo_social + 
                  costo_relocalizacion + 
                  costo_no_demanda, sense = pyo.minimize)

#RESTRICCIONES
#P puntos verdes
m.r1= pyo.constraint(exp=sum(m.y[j] for j in J) ==P)

#las unidades van a lo más a un punto verde

m.r2 = pyo.constraintlist()
for u in U:
    m.r2.add(expr = sum(m.x[u,j] for j in J) <=1)
             
# las personas no pueden viajar más que la distancia maxima
             
m.r3= pyo.ConstraintList()
for u in U:
    m.r3.add(expr=sum(dist[u][j]*m.x[u,j] for j in J) <= Dist_max)
             
 # no puede ir a una localizacion donde no este abierto el punto verde
m.r4= pyo.constraintlist()
for u in U:
    for j in J:
            m.r4.add(expr=m.x[u,j]<=m.y[j])
             
#la unidad territorial va al punto mas cercano
             
m.r5= pyo.constraintlist()
for u in U:
    for j in J:
            m.r5.add(expr=sum(m.x[u,j2] for j2 in J if dist[u,j] < dist[u,j2]) + m.y[j]<=1)
             
             




NameError: name 'u' is not defined

In [57]:
opt=pyo.SolverFactory ("glpk")
results= opt.solve(m)

print  (pyo.value(m.fo))

            

            

AttributeError: 'ConcreteModel' object has no attribute 'fo'